In [1]:
from flask import Flask, request, render_template, jsonify
import pandas as pd
import numpy as np
import re
import csv
import pickle

In [2]:
app = Flask(__name__, template_folder='templateFiles')

## API Untuk Input Teks

In [3]:
@app.route('/')
def my_form():
    return render_template('text_sentiment.html')

In [4]:
@app.route('/', methods=['POST'])
def my_form_post():
    text = request.form['text']
    count_vect = pickle.load(open('count_vect.pkl','rb'))
    model = pickle.load(open('model_neural_network.pkl', 'rb'))

    text = text.replace("\\n", " ")
    text = re.sub(r"(#)(.*)()", r"", text)
    text = re.sub(r"[!\"#$%&'()*+-./:;<=>?@[\]^_`{|}~]", r"", text)
    text = re.sub(r"(\s)(\1+)", r"\1", text)
    text = text.replace("RT ","")
    text = text.replace("USER ","")
    text = text.replace("USER","")
    text = re.sub(r"([a-z])(\1{3,})", r"\1\1", text)
    text = re.sub(r"(\\x)([a-z0-9]{2})", r"", text)
    text  = re.sub(r"(\\x8)", r"", text)
    text = text.lower()
    text = text.strip()
    
    preprocess_text = text
    
    predict_text = count_vect.transform([preprocess_text])
    result = model.predict(predict_text)[0]

    response = {
    'data' : preprocess_text,
    'result sentiment' : result
    }
    
    response_data = response
    return response_data

## API Untuk Input File

In [5]:
@app.route('/csv-upload', methods=['GET', 'POST'])
def index():
    return render_template('index.html')

In [6]:
def preprocessing_text(text): 
    text = text.replace("\\n", " ")
    text = re.sub(r"(#)(.*)()", r"", text)
    text = re.sub(r"[!\"#$%&'()*+-./:;<=>?@[\]^_`{|}~]", r"", text)
    text = re.sub(r"(\s)(\1+)", r"\1", text)
    text = text.replace("URL","")
    text = text.replace("RT ","")
    text = text.replace("USER ","")
    text = text.replace("USER","")
    text = re.sub(r"([a-z])(\1{3,})", r"\1\1", text)
    text = re.sub(r"(\\x)([a-z0-9]{2})", r"", text)
    text  = re.sub(r"(\\x8)", r"", text)
    text = text.lower()
    text = text.strip()
    return text

In [7]:
count_vect = pickle.load(open('count_vect.pkl','rb'))
model = pickle.load(open('model_neural_network.pkl', 'rb'))

def input_data(text):
    predict_text = count_vect.transform([text])
    result = model.predict(predict_text)[0]
    return result

C:\Users\crire\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\crire\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LabelBinarizer from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\crire\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator MLPClassifier from version 1.2.1 when usi

In [8]:
@app.route('/data', methods=['GET', 'POST'])
def data():
    if request.method == 'POST':
        file = request.form.get('csvfile')
        data = []
        data = pd.read_csv(file)
        
        data['process_text'] = data['Tweet'].apply(lambda x:preprocessing_text(x))
        data['predict'] = data['process_text'].apply(lambda x:input_data(x))
        
        data = data[['Tweet', 'process_text', 'predict']]
        
        return render_template('data.html', data=data.to_html(header=True, index=False))

In [9]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [10/Apr/2023 14:21:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2023 14:21:36] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [10/Apr/2023 14:21:41] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2023 14:22:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2023 14:23:01] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2023 14:23:28] "GET /csv-upload HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2023 14:23:33] "GET /csv-upload HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2023 14:23:52] "POST /data HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2023 14:24:02] "POST /data HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2023 14:24:31] "GET / HTTP/1.1" 200 -
